In [1]:
!pip install pinecone-client cohere sentence-transformers pdfplumber streamlit gradio


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import cohere
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = "17fa6302-1d73-4546-945f-8a8900066198"
cohere_api_key = "Tz1QAyJD5aIwdga6CJAxxjXCkp7cLdY1p9ZN3NC5"

# Initialize Pinecone
pc = Pinecone(api_key=pinecone_api_key)

# Initialize Cohere
co = cohere.Client(cohere_api_key)


c:\Users\bansa\AppData\Local\Programs\Python\Python312\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text


In [4]:
!pip install fpdf

In [41]:
from fpdf import FPDF

# Create instance of FPDF class
pdf = FPDF()

# Add a page
pdf.add_page()

# Set font
pdf.set_font("Arial", size=12)

# Add a cell
pdf.cell(200, 10, txt="Sample PDF Document", ln=True, align='C')

# Add some text
content = """
This is a sample PDF document created for testing the QA bot.
It contains multiple lines of text to simulate a document that might be uploaded
for question answering.

The purpose of this document is to provide a basic structure and content that
the QA bot can process, generate embeddings, and provide answers.

You can ask questions like:
1. What is the purpose of this document?
2. What is this document used for?

Feel free to test the document with your QA bot implementation.
"""

# Add the text content to the PDF
pdf.multi_cell(0, 10, content)

# Save the PDF with the name 'sample_document.pdf'
pdf.output("sample_document.pdf")

print("PDF created successfully.")

PDF created successfully.


In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

# Extract text and split into manageable chunks
def chunk_text(text, chunk_size=500):
    sentences = text.split('.')
    chunks = [' '.join(sentences[i:i+chunk_size]) for i in range(0, len(sentences), chunk_size)]
    return chunks

text = extract_text_from_pdf('data/sample_document.pdf')
chunks = chunk_text(text)

# Generate embeddings
embeddings = model.encode(chunks)

# Pinecone Setup
environment = ServerlessSpec(cloud='aws', region='us-east-1')

# Create or connect to the index
index_name = 'qa-bot'
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=len(embeddings[0]),  # Make sure this matches the embedding size
        metric='cosine',  # You can use 'euclidean' or 'cosine'
        spec=environment
    )

# Connect to the index
index = pc.Index(index_name)

# Upload embeddings
ids = [str(i) for i in range(len(embeddings))]
vectors_to_upsert = [
    {"id": ids[i], "values": embeddings[i].tolist(), "metadata": {"text": chunks[i]}}
    for i in range(len(embeddings))
]
index.upsert(vectors=vectors_to_upsert)


c:\Users\bansa\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: './data/sample_document.pdf'

In [6]:
def retrieve_relevant_chunks(query, top_k=3):
    query_embedding = model.encode([query])
    results = index.query(query_embedding, top_k=top_k)
    return [chunks[int(match['id'])] for match in results['matches']]

In [7]:
def generate_answer(retrieved_chunks, question):
    context = " ".join(retrieved_chunks)
    prompt = f"Context: {context}\n\nQuestion: {question}\nAnswer:"
    response = co.generate(model='xlarge', prompt=prompt)
    return response.generations[0].text


In [14]:
def answer_question(question):
    # Generate the embedding for the question
    question_embedding = model.encode([question])[0]

    # Perform the query using keyword arguments
    query_response = index.query(
        vector=question_embedding.tolist(),  # Convert the numpy array to a list
        top_k=5,  # Retrieve the top 5 most similar results
        include_metadata=True  # Include metadata in the results
    )

    # Check the response structure
    if 'matches' in query_response and len(query_response['matches']) > 0:
        top_match = query_response['matches'][0]
        if 'metadata' in top_match and 'text' in top_match['metadata']:
            context_text = top_match['metadata']['text']
        else:
            return "No relevant information found in the document."
    else:
        return "No relevant information found in the document."

    # Use Cohere to generate the answer with 'command' model
    response = co.generate(
        model='command',  # Use the 'command' model for free plan
        prompt=f"Question: {question}\nAnswer based on the following context: {context_text}",
        max_tokens=50,
        temperature=0.5
    )
    return response.generations[0].text.strip()


In [15]:
# Example query
question = "What is the main topic of the document?"
print(answer_question(question))


This document is a sample PDF used for testing Question Answering models. 

The document itself is primarily focused on providing a structured sample for testing, and does not contain any inherent topic or content beyond generic text.


In [16]:
!pip install streamlit